In [1]:
!pip install langchain langchain-community langchain-core langchain-groq faiss-cpu sentence-transformers docling


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.1/195.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.6/86.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━

In [2]:

import os
import warnings
from docling.document_converter import DocumentConverter
from pathlib import Path
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [3]:
# Document conversion
def load_and_convert_document(file_path):
    converter = DocumentConverter()
    result = converter.convert(file_path)
    return result.document.export_to_markdown()

source = "/content/amazon-10-q-q3-2024.pdf"
markdown_content = load_and_convert_document(source)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
# Splitting markdown content into chunks
def get_markdown_splits(markdown_content):
    headers_to_split_on = [("#", "Header 1"), ("##", "Header 2"), ("###", "Header 3")]
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on, strip_headers=False)
    return markdown_splitter.split_text(markdown_content)
chunks = get_markdown_splits(markdown_content)


In [5]:
# Embedding and vector store setup
def setup_vector_store(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    single_vector = embeddings.embed_query("this is some text data")
    index = faiss.IndexFlatL2(len(single_vector))
    vector_store = FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={}
    )
    vector_store.add_documents(documents=chunks)
    return vector_store

vector_store = setup_vector_store(chunks)

# Setup retriever
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={'k': 3})


/tmp/ipython-input-5-1714052293.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [6]:

# Formatting documents for RAG
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [7]:
# Setting up the RAG chain
def create_rag_chain(retriever):
    prompt = """
        You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
        If you don't know the answer, just say that you don't know.
        Answer in bullet points. Make sure your answer is relevant to the question and it is answered from the context only.
        Question: {question}
        Context: {context}
        Answer:
    """
    model = ChatGroq(model_name="llama3-8b-8192", api_key = "")  # ✅ Choose a Groq-supported model
    prompt_template = ChatPromptTemplate.from_template(prompt)
    return (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt_template
        | model
        | StrOutputParser()
    )



In [9]:
# Create RAG chain
try:
    # You must provide your Groq API key here
    rag_chain = create_rag_chain(retriever)

    questions = [
        "What was Amazon's total revenue in Q3 2024?",
        "How does the revenue in Q3 2024 compare to Q3 2023?",
        "What was the net income for Q3 2024, and how does it compare year over year?",
        "What were the earnings per share (basic and diluted) in Q3 2024?",
        "How much did Amazon earn from product vs. service sales?",
        "What were the main operating expense categories and their values in Q3 2024?",
        "What was Amazon's operating income in Q3 2024?",
    ]

    if rag_chain:
        for question in questions:
            print(f"Question: {question}")
            try:
                for chunk in rag_chain.stream(question):
                    print(chunk, end="", flush=True)
                print("\n" + "-" * 50 + "\n")
            except Exception as e:
                print(f"Error answering question: {str(e)}")
                print("\n" + "-" * 50 + "\n")
    else:
        print("Failed to create RAG chain. Please check your API key and try again.")

except Exception as e:
    print(f"Error initializing RAG chain: {str(e)}")
    print("Please make sure you have:")
    print("1. A valid Groq API key set in the create_rag_chain function")
    print("2. The groq package installed (pip install groq)")
    print("3. A stable internet connection")

Error initializing RAG chain: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable
Please make sure you have:
1. A valid Groq API key set in the create_rag_chain function
2. The groq package installed (pip install groq)
3. A stable internet connection


In [10]:
def create_rag_chain(retriever):
    if not retriever:
        return None
    try:
        prompt = """
            You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
            If you don't know the answer, just say that you don't know.
            Answer in bullet points. Make sure your answer is relevant to the question and it is answered from the context only.
            Question: {question}
            Context: {context}
            Answer:
        """
        # Replace with your actual API key
        model = ChatGroq(model_name="llama3-8b-8192", api_key="your-actual-key-here")
        prompt_template = ChatPromptTemplate.from_template(prompt)
        return (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | model
            | StrOutputParser()
        )
    except Exception as e:
        print(f"Error creating RAG chain: {str(e)}")
        return None

In [13]:
# Create RAG chain
def create_rag_chain(retriever):
    if not retriever:
        return None

    prompt = """
        You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
        If you don't know the answer, just say that you don't know.
        Answer in bullet points. Make sure your answer is relevant to the question and it is answered from the context only.
        Question: {question}
        Context: {context}
        Answer:
    """

    try:
        model = ChatGroq(
            model_name="llama3-8b-8192",
            api_key="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"  # Your API key inserted here
        )

        prompt_template = ChatPromptTemplate.from_template(prompt)
        return (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | model
            | StrOutputParser()
        )
    except Exception as e:
        print(f"Error creating RAG chain: {str(e)}")
        return None

# Now create and use the RAG chain
try:
    rag_chain = create_rag_chain(retriever)
    if not rag_chain:
        raise ValueError("Failed to create RAG chain. Please check your API key and network connection.")

    questions = [
        "What was Amazon's total revenue in Q3 2024?",
        "How does the revenue in Q3 2024 compare to Q3 2023?",
        "What was the net income for Q3 2024, and how does it compare year over year?",
        "What were the earnings per share (basic and diluted) in Q3 2024?",
        "How much did Amazon earn from product vs. service sales?",
        "What were the main operating expense categories and their values in Q3 2024?",
        "What was Amazon's operating income in Q3 2024?",
    ]

    for question in questions:
        print(f"\nQuestion: {question}")
        try:
            response = ""
            for chunk in rag_chain.stream(question):
                response += chunk
            print(response)
        except Exception as e:
            print(f"Error answering question: {str(e)}")
        print("-" * 50)

except Exception as e:
    print(f"Initialization error: {str(e)}")
    print("\nTroubleshooting steps:")
    print("1. Verify your API key is correct and active")
    print("2. Check your internet connection")
    print("3. Ensure you haven't exceeded rate limits")
    print("4. Try a simpler question first to test the connection")


Question: What was Amazon's total revenue in Q3 2024?
Here is the answer to the question:

* The context does not mention Amazon's total revenue in Q3 2024. The provided information only includes revenue guidance for the fourth quarter of 2024, which is between $181.5 billion and $188.5 billion.
* The total revenue for the nine months ended September 30, 2024 is not mentioned in the context.
* The context does not provide Amazon's total revenue in Q3 2024, so I don't know the answer to this question.
--------------------------------------------------

Question: How does the revenue in Q3 2024 compare to Q3 2023?
Here is the answer to the question:

• The question is asking how the revenue in Q3 2024 compares to Q3 2023, but the provided context only includes guidance for the Fourth Quarter 2024, not Q3 specifically.
• The context does not provide information on Q3 2023, so it is not possible to compare the revenue in Q3 2024 to Q3 2023.
• Therefore, the answer is: I don't know.
------

In [ ]:
# Main execution logic
if __name__ == "__main__":
    # Load document
    source = "/content/amazon-10-q-q3-2024.pdf"
    markdown_content = load_and_convert_document(source)
    chunks = get_markdown_splits(markdown_content)

    # Create vector store
    vector_store = setup_vector_store(chunks)

    # Setup retriever
    retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={'k': 3})

    # Create RAG chain
    rag_chain = create_rag_chain(retriever)

    # Questions for retrieval
    # Questions for retrieval (Amazon-specific)
    questions = [
    # Financial Performance
    "What was Amazon’s total revenue in Q3 2024?",
    "How does the revenue in Q3 2024 compare to Q3 2023?",
    "What was the net income for Q3 2024, and how does it compare year over year?",
    "What were the earnings per share (basic and diluted) in Q3 2024?",
    "How much did Amazon earn from product vs. service sales?",
    "What were the main operating expense categories and their values in Q3 2024?",
    "What was Amazon’s operating income in Q3 2024?",
    ]



    # Answer questions
    for question in questions:
        print(f"Question: {question}")
        for chunk in rag_chain.stream(question):
            print(chunk, end="", flush=True)
        print("\n" + "-" * 50 + "\n")

In [ ]:
import gradio as gr

# Global variable to store vector store and RAG chain after file upload
vector_store = None
rag_chain = None

def upload_and_process(file):
    global vector_store, rag_chain
    file_path = file.name
    markdown_content = load_and_convert_document(file_path)
    chunks = get_markdown_splits(markdown_content)
    vector_store = setup_vector_store(chunks)
    retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={'k': 3})
    rag_chain = create_rag_chain(retriever)
    return "✅ Document processed and RAG chain created. You can now ask questions."

def ask_question(question):
    if rag_chain is None:
        return "❌ Please upload and process a document first."
    response = ""
    for chunk in rag_chain.stream(question):
        response += chunk
    return response

# Launch Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("## 📊 Amazon 10-Q RAG-Based QA System using Groq + FAISS")

    with gr.Row():
        with gr.Column():
            file_input = gr.File(label="📁 Upload Amazon 10-Q PDF", file_types=[".pdf"])
            upload_btn = gr.Button("🔄 Upload & Process")
            upload_output = gr.Textbox(label="Processing Status", lines=2)

        with gr.Column():
            question_input = gr.Textbox(label="❓ Ask a Question", placeholder="e.g. What was Amazon’s total revenue in Q3 2024?")
            ask_btn = gr.Button("🔍 Get Answer")
            answer_output = gr.Textbox(label="📢 Answer", lines=10)

    upload_btn.click(fn=upload_and_process, inputs=[file_input], outputs=[upload_output])
    ask_btn.click(fn=ask_question, inputs=[question_input], outputs=[answer_output])

demo.launch(

In [ ]:
# Main execution logic
if __name__ == "__main__":
    # Load document
    source = "/content/amazon-10-q-q3-2024.pdf"
    markdown_content = load_and_convert_document(source)
    chunks = get_markdown_splits(markdown_content)

    # Create vector store
    vector_store = setup_vector_store(chunks)

    # Setup retriever
    retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={'k': 3})

    # Create RAG chain
    rag_chain = create_rag_chain(retriever)

    # Questions for retrieval
    # Questions for retrieval (Amazon-specific)
    questions = [
    # Financial Performance
    "What was Amazon’s total revenue in Q3 2024?",
    "How does the revenue in Q3 2024 compare to Q3 2023?",
    "What was the net income for Q3 2024, and how does it compare year over year?",
    "What were the earnings per share (basic and diluted) in Q3 2024?",
    "How much did Amazon earn from product vs. service sales?",
    "What were the main operating expense categories and their values in Q3 2024?",
    "What was Amazon’s operating income in Q3 2024?",

    # Segment Analysis
    "What was the revenue and operating income for Amazon Web Services (AWS) in Q3 2024?",
    "How did the North America and International segments perform in terms of revenue and profit?",
    "Which segment contributed the most to Amazon’s operating income?",

    # Cash Flow & Capital Expenditures
    "How much cash did Amazon generate from operating activities?",
    "What were the major uses of cash in investing and financing activities?",
    "What were Amazon’s capital expenditures in Q3 2024?",

    # Debt and Liabilities
    "What is Amazon’s total long-term debt as of September 30, 2024?",
    "What are the upcoming debt maturities and interest rates for Amazon's outstanding bonds?",

    # Stock & Shareholder Equity
    "What were the changes in stockholders’ equity this quarter?",
    "Did Amazon repurchase any stock in Q3 2024?",

    # Other Financial Indicators
    "What are the values of accounts receivable, inventories, and marketable securities as of September 30, 2024?",
    "What were Amazon’s total assets and liabilities?",

    # Legal and Regulatory
    "What major legal proceedings or lawsuits was Amazon involved in during Q3 2024?",
    "What was the outcome of the Kove IO patent case?",
    "Are there any significant antitrust or consumer protection investigations ongoing?",

    # Taxation and Accounting
    "What is Amazon’s effective tax rate for Q3 2024?",
    "What discrete tax items impacted Amazon’s tax expense?",
    "Are there any significant tax disputes or contingencies noted?",

    # Revenue Breakdown
    "What was the revenue breakdown by business line (Online stores, AWS, advertising, etc.)?",
    "How did subscription services and advertising revenue perform compared to last year?"
]



    # Answer questions
    for question in questions:
        print(f"Question: {question}")
        for chunk in rag_chain.stream(question):
            print(chunk, end="", flush=True)
        print("\n" + "-" * 50 + "\n")

